Project #5: Video Stitching and Processing 

## CS445: Computational Photography - Spring 2020

### Setup


In [ ]:
!pip uninstall opencv-python -y
# downgrade OpenCV a bit to use SIFT
# !pip install opencv-contrib-python==3.4.2.17 --force-reinstall
!pip install ffmpeg-python # for converting to video

import ffmpeg
import cv2
import numpy as np
import os
from numpy.linalg import svd, inv
import utils
%matplotlib inline
from matplotlib import pyplot as plt
from typing import List, Any

In [ ]:
# modify to where you store your project data including utils
datadir = "/home/abot/cs445_comp_photo/cs445_a5" 

utilfn = os.path.join(datadir, "utils.py")
!cp "$utilfn" .
imagesfn = os.path.join(datadir, "images")
!cp -r "$imagesfn" .

In [ ]:
np.random.seed(182736745)

### Part I: Stitch two key frames 

#### This involves:
1. compute homography H between two frames; 
2. project each frame onto the same surface;
3. blend the surfaces.

Check that your homography is correct by plotting four points that form a square in frame 270 and their projections in each image.

In [ ]:
def score_projection(pt1, pt2):
  '''
  Score corresponding to the number of inliers for RANSAC
  Input: pt1 and pt2 are 2xN arrays of N points such that pt1[:, i] and pt2[:,i] should
          be close in Euclidean distance if they are inliers
  Outputs: score (scalar count of inliers) and inliers (1xN logical array)
  '''
  kThreshold = 3.0
  _, N = pt1.shape
  
  euclid_dist = np.sqrt(np.sum(np.power(pt1 - pt2, 2), axis=0))
  # (N,) -> (1,N)
  euclid_dist = euclid_dist[np.newaxis, :]
  assert(euclid_dist.shape == (1,N))
  inliers = euclid_dist < kThreshold
  assert(inliers.shape == (1,N))
  score = np.sum(inliers)
  
  return score, inliers


def auto_homography(Ia,Ib, homography_func=None,normalization_func=None):
    '''
    Computes a homography that maps points from Ia to Ib

    Input: Ia and Ib are images
    Output: H is the homography

    '''
    if Ia.dtype == 'float32' and Ib.dtype == 'float32':
        Ia = (Ia*255).astype(np.uint8)
        Ib = (Ib*255).astype(np.uint8)
    
    Ia_gray = cv2.cvtColor(Ia,cv2.COLOR_BGR2GRAY)
    Ib_gray = cv2.cvtColor(Ib,cv2.COLOR_BGR2GRAY)

    # Initiate SIFT detector
    sift = cv2.xfeatures2d.SIFT_create()
    
    # find the keypoints and descriptors with SIFT
    kp_a, des_a = sift.detectAndCompute(Ia_gray,None)
    kp_b, des_b = sift.detectAndCompute(Ib_gray,None)    
    
    # BFMatcher with default params
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des_a,des_b, k=2)

    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append(m)
   
    numMatches = int(len(good))

    matches = good

    # Xa and Xb are 3xN matrices that contain homogeneous coordinates for the N
    # matching points for each image
    Xa = np.ones((3,numMatches))
    Xb = np.ones((3,numMatches))
    
    for idx, match_i in enumerate(matches):
        Xa[:,idx][0:2] = kp_a[match_i.queryIdx].pt
        Xb[:,idx][0:2] = kp_b[match_i.trainIdx].pt

    ## RANSAC
    niter = 1000
    best_score = 0
    n_to_sample = 4 # Put the correct number of points here

    for t in range(niter):
        # estimate homography
        subset = np.random.choice(numMatches, n_to_sample, replace=False)
        pts1 = Xa[:,subset]
        pts2 = Xb[:,subset]
        
        H_t = homography_func(pts1, pts2, normalization_func) # edit helper code below (computeHomography)

        
        # score homography
        Xb_ = np.dot(H_t, Xa) # project points from first image to second using H
        
        score_t, inliers_t = score_projection(Xb[:2,:]/Xb[2,:], Xb_[:2,:]/Xb_[2,:])

        if score_t > best_score:
            best_score = score_t
            H = H_t
            in_idx = inliers_t
    
    print('best score: {:02f}'.format(best_score))

    # Optionally, you may want to re-estimate H based on inliers

    return H

In [ ]:
def computeHomography(pts1, pts2, normalization_func=None):
    '''
    Compute homography that maps from pts1 to pts2 using SVD. Normalization is optional.
     
    Input: pts1 and pts2 are 3xN matrices for N points in homogeneous
    coordinates. 
    
    Output: H is a 3x3 matrix, such that pts2~=H*pts1
    '''
    kNumCols = 9
    three, N = pts1.shape
    assert(three == 3)
    assert(N == 4)
    orig_pts1 = pts1.copy()
    orig_pts2 = pts2.copy()
    
    # Normalize the points (x) matrices
    if normalization_func:
      pts1, pts2, T, TP = normalization_func(pts1, pts2)
  
    # Aliases for indices that match lecture nomenclature. Homo coords [u, v, w]
    u, v = [0, 1]
    up, vp = [0, 1]
    
    A = np.zeros((2*N, kNumCols))
    A[::2,0] = -pts1[u, :]  # 0th col
    A[::2,1] = -pts1[v, :]  # 1st col
    A[::2,2] = -1.0         # 2nd col
    # next 3 cols are 0.
    A[::2,6] = pts1[u , :] * pts2[up, :]  # 6th col
    A[::2,7] = pts1[v , :] * pts2[up, :]  # 7th col
    A[::2,8] = pts2[up, :]                # 8th col
    
    # 1st 3 cols are 0.
    A[1::2,3] = -pts1[u, :]  # 3rd col
    A[1::2,4] = -pts1[v, :]  # 4th col
    A[1::2,5] = -1.0         # 5th col
    A[1::2,6] = pts1[u , :] * pts2[vp, :]  # 6th col
    A[1::2,7] = pts1[v , :] * pts2[vp, :]  # 7th col
    A[1::2,8] = pts2[vp, :]                # 8th col
   
    # S is sorted in descending order. diag(S)*V = (K,K) * (K,N) so last col is multiplied by smallest singular value.
    U, S, Vh = np.linalg.svd(A, compute_uv=True)
    h = Vh[-1, :]
    H = h.copy().reshape((three, three), order='C')
    assert(h[3] == H[1,0])  # Check reshape index correct.
    assert(h[5] == H[1,2])
    
    if normalization_func:
      H = TP @ H @ T

    # For scaling of H matrix by the w', or lamba homo coord see:
    # https://math.stackexchange.com/questions/494238/how-to-compute-homography-matrix-h-from-corresponding-points-2d-2d-planar-homog
    unscaled_homo = H @ pts1
    scaled_homo = unscaled_homo / unscaled_homo[-1, :]
    if not np.allclose(pts2, scaled_homo, rtol=0.01):
      print('WARNING:')
      print(f'pts1\n {pts1}')
      print(f'pts2\n {pts2}')
      print(f'H*pts1 \n{unscaled_homo}\n')
      print(f'pts2 - H*pts1 diff {pts2 - scaled_homo}\n')
    assert(np.allclose(pts2, scaled_homo, rtol=0.01))
    return H

def normalizeHomography(pts1, pts2):
  ''' Normalizes the x vectors in the equation x_p = H @ x
  
  Input: pts1 and pts2 are 3xN matrices for N points in homogeneous
  coordinates. 
  
  Output:
    pts1_p: transformed (0 mean, unit variance) version of pts1
    pts2_p: transformed (0 mean, unit variance) version of pts2
    T: the transform corresponding to pts1
    TP: the transform corresponding to pts2:
  
  We compute the homography on the transformed H so all points are the same scale
  around ~1. To inver the transform and recover original, unnormalized H, we can
  do the following:
      H = TP @ HP @T, 
 
  pts2~=H*pts1, so pts2 is x_p, and pts1 is x.
  
  ~x = T @ x
  ~x_p = Tp @ x_p
  
  Where ~x, and ~x_p are transformed versions of x and x_p with ~0 mean and unit
  variance.
  '''
  T = np.diag([1.0,1.0,1.0])
  TP = np.diag([1.0,1.0,1.0])
  sigma_T = np.diag([np.std(pts1[0,:]), np.std(pts1[1,:]), 1.0])   # x,y,1
  sigma_TP = np.diag([np.std(pts2[0,:]), np.std(pts2[1,:]), 1.0])  # x,y,1
  mu_x, mu_y   = (np.mean(pts1[0,:]), np.mean(pts1[1,:]))  # x,y
  mu_xp, mu_yp = (np.mean(pts2[0,:]), np.mean(pts2[1,:]))  # x,y
  
  T[0, -1] = -mu_x
  T[1, -1] = -mu_y
  TP[0, -1] = -mu_xp
  TP[1, -1] = -mu_yp
  
  T = sigma_T @ T
  TP = sigma_TP @ TP
  
  pts1_p = T @ pts1
  pts2_p = TP @ pts2
  
  return pts1_p, pts2_p, T, TP

In [ ]:
# images location
im1 = './images/input/frames/f0270.jpg'
im2 = './images/input/frames/f0450.jpg'

# Load an color image in grayscale
im1 = cv2.imread(im1)
im2 = cv2.imread(im2)

H = auto_homography(im1,im2, computeHomography, normalizeHomography)
print(H/H.max()) 

# plot the frames here
box_pts = np.array([[300, 400, 400, 300, 300], [100, 100, 200, 200, 100], [1, 1, 1, 1, 1]])
plt.figure()
plt.imshow(im1[:,:,[2,1,0]])
plt.plot(box_pts[0,:], box_pts[1, :], 'r-')

# TO DO: project points into im2 and display the projected lines on im2
unscaled_homo = H @ box_pts
scaled_homo = unscaled_homo / unscaled_homo[-1, :]
plt.figure()
plt.imshow(im2[:,:,[2,1,0]])
plt.plot(scaled_homo[0,:], scaled_homo[1, :], 'r-')



In [ ]:
projectedWidth = 1600
projectedHeight = 500
Tr = np.array([[1, 0, 660], [0, 1, 120], [0, 0, 1]]).astype(np.float32)
canvas = np.zeros((projectedHeight, projectedWidth))

# Projects im1 and im2 according to T1 and T2 to an image of size WxH and then
# blends the projected images by filling any zero values in projIm2 with values
# from projIm1
projIm1 = cv2.warpPerspective(im1, np.dot(Tr,H), (projectedWidth, projectedHeight))
projIm2 = cv2.warpPerspective(im2, Tr, (projectedWidth, projectedHeight))
blendOut = utils.blendImages(projIm1, projIm2) 

plt.figure(figsize=(25,20))
plt.imshow(blendOut[:,:,[2,1,0]])


### Part II: Panorama using five key frames

Produce a panorama by mapping five key frames [90, 270, 450, 630, 810] onto the same reference frame 450.  


In [ ]:
key_frames_idx = np.array([90, 270, 450, 630, 810])-1

frames = np.zeros((len(key_frames_idx), im1.shape[0], im1.shape[1], im1.shape[2]),dtype='uint8')
for n in range(len(key_frames_idx)):
  frames[n] = cv2.imread("./images/input/frames/f0{num}.jpg".format(num=str(key_frames_idx[n]+1).zfill(3)))

def warp_frames(frames: List[np.ndarray]) -> np.ndarray:
  projectedWidth = 1600
  projectedHeight = 500
  Tr = np.array([[1, 0, 660], [0, 1, 120], [0, 0, 1]]).astype(np.float32)
  canvas = np.zeros((projectedHeight, projectedWidth, 3))
  
  # We reference everything to the 450th frame. Add 1 so we keep the black
  # pixels when using blendImages.
  kRefFrame = frames[len(frames)//2].copy()
  kRefFrame[kRefFrame == 0] = 1

  for f in frames:
    H = auto_homography(f, kRefFrame, computeHomography)
    p1 = cv2.warpPerspective(f, np.dot(Tr,H), (projectedWidth, projectedHeight))
    p_ref = cv2.warpPerspective(kRefFrame, Tr, (projectedWidth, projectedHeight))
    blendOut = utils.blendImages(p1, p_ref)  # Ref is second argument.
    print(f'p1 shape {p1.shape}, p_ref shape {p_ref.shape}')
    print(f'blendOut {blendOut.shape}, canvas {canvas.shape}')
    canvas = utils.blendImages(blendOut, canvas)
    plt.figure(figsize=(25,20))
    plt.imshow(blendOut[:,:,[2,1,0]])
    
  return canvas

canvas = warp_frames(frames)
plt.figure(figsize=(25,20))
plt.imshow(canvas[:,:,[2,1,0]])

### Part 3: Map the video to the reference plane

Project each frame onto the reference frame (using same size panorama) to create a video that shows the portion of the panorama revealed by each frame

In [ ]:
# read all the images
import os 
dir_frames = 'images/input/frames'
filenames = []
filesinfo = os.scandir(dir_frames)

filenames = [f.path for f in filesinfo if f.name.endswith(".jpg")]
filenames.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

frameCount = len(filenames)
frameHeight, frameWidth, frameChannels = cv2.imread(filenames[0]).shape
frames = np.zeros((frameCount, frameHeight, frameWidth, frameChannels),dtype='uint8')

for idx, file_i in enumerate(filenames):
  frames[idx] = cv2.imread(file_i)



In [ ]:
# TO DO part 3 solution

# create your video (see tips)

### Part 4: Create background panorama

Create a background panorama based on the result from Part 3.


In [ ]:
# TO DO part 4

### Part 5: Create background movie

Generate a movie that looks like the input movie but shows only background pixels. For each frame of the movie, you need to estimate a projection from the panorama to that frame. Your solution can use the background image you created in Part 4 and the per-frame homographies you created in Part 3. 


In [ ]:
# TO DO part 5



### Part 6: Create foreground movie

In the background video, moving objects are removed. In each frame, those pixels that are different enough than the background color are considered foreground. For each frame determine foreground pixels and generate a movie that emphasizes or includes only foreground pixels.

In [ ]:
# TO DO part 6


## Bells and whistles